# 1. Import Packages

In [1]:
# Importing all required packages at the start of the notebook
import os
import matplotlib.pyplot as plt
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import seaborn as sns
from scipy.stats import shapiro, kruskal, f_oneway

# 2. Data Directionary

In [3]:
# Location
data_dir = "Project_data/Differential_Abundance"
! mkdir -p "$data_dir"

In [29]:
# Paths to project inputs
input_table    = "Project_data/Import_and_Denoising/dada2_table.qza"
input_taxonomy = "Project_data/Taxonomy/taxonomy_pretrained.qza"
input_metadata = "Project_data/Metadata/updated_fungut_metadata.tsv"

# 3. Differential Abundance - IBD Status

In [12]:
# Filter features: keep only reasonably frequent ASVs
! qiime feature-table filter-features \
  --i-table $input_table \
  --p-min-frequency 25 \
  --p-min-samples 4 \
  --o-filtered-table $data_dir/table_abund.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: Project_data/Differential_Abundance/table_abund.qza


In [43]:
# Collapse to species level (L7)
! qiime taxa collapse \
  --i-table $data_dir/table_abund.qza \
  --i-taxonomy $input_taxonomy \
  --p-level 7 \
  --o-collapsed-table $data_dir/table_abund_L7.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: Project_data/Differential_Abundance/table_abund_L7.qza


In [52]:
# ANCOM-BC: effect of IBD
! qiime composition ancombc \
  --i-table $data_dir/table_abund_L7.qza \
  --m-metadata-file $input_metadata \
  --p-formula "ibd_sample" \
  --o-differentials $data_dir/ancombc_ibd_L7_diffs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: Project_data/Differential_Abundance/ancombc_ibd_L7_diffs.qza


In [53]:
# Barplot results
! qiime composition da-barplot \
  --i-data $data_dir/ancombc_ibd_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_ibd_L7_barplot.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_ibd_L7_barplot.qzv


In [32]:
! qiime composition tabulate \
  --i-data $data_dir/ancombc_ibd_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_ibd_L7_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_ibd_L7_results.qzv


In [37]:
Visualization.load("Project_data/Differential_Abundance/ancombc_ibd_L7_barplot.qzv")

<visualization: Visualization uuid: f8bb0bda-6b6e-4771-a7d1-5dbc611a8fa3>

In [38]:
Visualization.load("Project_data/Differential_Abundance/ancombc_ibd_L7_results.qzv")

<visualization: Visualization uuid: 3dd2b3e0-bf42-4b63-9677-42c1c1a5d6b2>

# 4. Differential Abundance - Gluten Status

In [55]:
# because there were "/" in the gluten column
meta = pd.read_csv(input_metadata, sep="\t")

# create a cleaned version of the gluten column without '/'
meta["gluten_clean"] = meta["gluten_sample"].str.replace("/", "_", regex=False)

meta_clean_path = "Project_data/Differential_Abundance/metadata_gluten_clean.tsv"
meta.to_csv(meta_clean_path, sep="\t", index=False)

meta_clean_path

'Project_data/Differential_Abundance/metadata_gluten_clean.tsv'

In [56]:
# ANCOM-BC: effect of Gluten
! qiime composition ancombc \
  --i-table $data_dir/table_abund_L7.qza \
  --m-metadata-file Project_data/Differential_Abundance/metadata_gluten_clean.tsv \
  --p-formula "gluten_clean" \
  --o-differentials $data_dir/ancombc_gluten_L7_diffs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: Project_data/Differential_Abundance/ancombc_gluten_L7_diffs.qza


In [57]:
# Barplot results
! qiime composition da-barplot \
  --i-data $data_dir/ancombc_gluten_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_gluten_L7_barplot.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_gluten_L7_barplot.qzv


In [58]:
! qiime composition tabulate \
  --i-data $data_dir/ancombc_gluten_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_gluten_L7_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_gluten_L7_results.qzv


In [59]:
Visualization.load("Project_data/Differential_Abundance/ancombc_gluten_L7_barplot.qzv")

<visualization: Visualization uuid: a65745c4-2eab-4e66-aba2-0681762eb574>

In [60]:
Visualization.load("Project_data/Differential_Abundance/ancombc_gluten_L7_results.qzv")

<visualization: Visualization uuid: 42662fcb-d5b3-4a49-8c21-bc221aa628f8>

# 5. Differential Abundance - Diet

In [63]:
# because there were "/" in the gluten column
meta = pd.read_csv(input_metadata, sep="\t")
meta["diet_type_sample"].unique()

meta["diet_clean"] = (
    meta["diet_type_sample"]
    .str.replace("/", "_", regex=False)
    .str.replace(" ", "_", regex=False)
)
clean_meta_path = "Project_data/Differential_Abundance/metadata_diet_clean.tsv"
meta.to_csv(clean_meta_path, sep="\t", index=False)

clean_meta_path

'Project_data/Differential_Abundance/metadata_diet_clean.tsv'

In [64]:
# ANCOM-BC: effect of Diet
! qiime composition ancombc \
  --i-table $data_dir/table_abund_L7.qza \
  --m-metadata-file Project_data/Differential_Abundance/metadata_diet_clean.tsv \
  --p-formula "diet_clean" \
  --o-differentials $data_dir/ancombc_diet_L7_diffs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: Project_data/Differential_Abundance/ancombc_diet_L7_diffs.qza


In [65]:
# Barplot results
! qiime composition da-barplot \
  --i-data $data_dir/ancombc_diet_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_diet_L7_barplot.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_diet_L7_barplot.qzv


In [66]:
! qiime composition tabulate \
  --i-data $data_dir/ancombc_diet_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_diet_L7_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_diet_L7_results.qzv


In [67]:
Visualization.load("Project_data/Differential_Abundance/ancombc_diet_L7_barplot.qzv")

<visualization: Visualization uuid: 3691da94-dc59-4cb2-a7bc-671bd79ead13>

In [68]:
Visualization.load("Project_data/Differential_Abundance/ancombc_diet_L7_results.qzv")

<visualization: Visualization uuid: c053f40b-3b43-4d7e-ac10-5a7cbbeab185>

# 6. Differential Abundance - Gender

In [69]:
# ANCOM-BC: effect of Sex
! qiime composition ancombc \
  --i-table $data_dir/table_abund_L7.qza \
  --m-metadata-file $input_metadata \
  --p-formula "sex_sample" \
  --o-differentials $data_dir/ancombc_sex_L7_diffs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: Project_data/Differential_Abundance/ancombc_sex_L7_diffs.qza


In [70]:
# Barplot results
! qiime composition da-barplot \
  --i-data $data_dir/ancombc_sex_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_sex_L7_barplot.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_sex_L7_barplot.qzv


In [71]:
! qiime composition tabulate \
  --i-data $data_dir/ancombc_sex_L7_diffs.qza \
  --o-visualization $data_dir/ancombc_sex_L7_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Differential_Abundance/ancombc_sex_L7_results.qzv


In [72]:
Visualization.load("Project_data/Differential_Abundance/ancombc_sex_L7_barplot.qzv")

<visualization: Visualization uuid: 2148be1b-8bab-4756-8647-4b4332fce385>

In [73]:
Visualization.load("Project_data/Differential_Abundance/ancombc_sex_L7_results.qzv")

<visualization: Visualization uuid: 9d4cf4f3-d00c-4f00-9bb5-b81c4066e3ca>